# Platform Positioning

In [1]:
from __future__ import print_function

In [2]:
import random as random #For random initial platform position
import numpy as np
import pandas as pd

import geopandas as gpd
from shapely.geometry.polygon import Polygon
from shapely.geometry import Point, LineString 
import shapely
from geopy import distance # For calculating distance between two points 


import pickle  as pkl # For importing pickled objects

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from math import tan, isclose, pi #F or calculating point on circumference - step out radius

# Ingest Processed Data
### Target Polygons
These were processed in the preceeding notebook and stored in '1_processed'

In [3]:
with open('data/1_processed/target_polygon_gdf.pkl', 'rb') as f:
    gdf_targPoly = pkl.load(f)
    
gdf_targPoly.plot()

### Well Concepts
As no suitable test data exists, dummy data is generated. There is a separate dummdatacreator.py file containing this work.

Rather than have Well Concepts randomly placed anywhere, we restrict it to only be within Target Polygon space

In [4]:
from dummydatacreator import  GenerateMultipleWellConceptsTEST as GenerateWCs # This is not supported in Azure notebooks

In [5]:
WC_name, WC_points, WC_resource = GenerateWCs(gdf_targPoly)

print("Well Concepts Generated:\n",'\n'.join('Name: {}, Points: {}, Resource: {}'.format(WC_name[k], WC_points[k], WC_resource[k]) for k in range(5)))

Well Concepts Generated:
 Name: N-6, Points: [-60.4313541533854, 10.033255504987698], Resource: 44.86212412803017
Name: Z-4, Points: [-60.387191278059305, 10.055110283713734], Resource: 93.74586540323433
Name: X-17, Points: [-60.385824446250865, 10.053160038625931], Resource: 207.41733359833688
Name: Uc-15, Points: [-60.30709401430253, 10.099520579491553], Resource: 37.3597806864533
Name: I-2, Points: [-60.306429961145476, 10.06058785513539], Resource: 203.088265058085


# Mapping
In order to construct the map containing Targets, Well Concepts, Platforms, some functions are required:


### Platform Step-Out Radius Functions
As there is no method for deriving the circumference around a point in EPSG:4326, functions are required to calculate this.
- A function to find a single point on the circumference (within a given tolerance)
- A function to find multiple points all around the circumference, sufficient for plotting the circle

In [6]:
def GetXYatCircumference(centre, desired_radius, theta, tolerance = 0.00001):
    '''Calculates the X,Y coordinates of a point on the circumference of a circle, 
    with a given desired radius and angle to the vertical.
    
    Theta in radians
    '''
    
    #Initialise x,y coordinates
    x = centre[0]
    y = centre[1]

    
    #Set initial increment values
        #If we are working in the lower two quadrants, i.e. where y will be less than centre y
    if theta > pi/2 and theta < (3*pi)/2:
        y_increment=-0.01
    else:
        y_increment=0.01

        
    #Loop until the radius of the point and centre, is close to the desired radius
    while isclose(distance.distance(centre, [x,y]).kilometers,  desired_radius ,rel_tol=0.00001) == False:

        # Add the increment whilst the distance is less than desired
        while distance.distance(centre, [x,y]) < desired_radius:
            x_increment = y_increment * tan(theta)
            x += x_increment
            y += y_increment

        # Subtract the increment whilst the distance is more than desired
        while distance.distance(centre, [x,y]) > desired_radius:
            x_increment = y_increment * tan(theta)
            x -= x_increment
            y -= y_increment

        #Reduce the size of the increment to increase precision  
        y_increment = y_increment/4

    return [x,y]

In [7]:
def GetCircumferenceXY(centre, radius, n_points, tolerance = 0.00001):
    '''Calculates x,y coordinates of a specified number points on the circumference of a 
    circle with a specified centre    
    
    NB: n_points must odd, to ensure we do not get a trig erro
    '''
    assert n_points % 2 != 0, "n_points must not be an even number - gives a trig error"

    # The angle between each radius        
    theta_increment = (2*pi)/n_points

    # The angles of all the radii we will use
    thetas = [theta_increment * i for i in range(n_points)]
    
    # Get a list of all calculated XYs on the circumference
    xys = [GetXYatCircumference(centre, radius,theta_i,tolerance) for theta_i in  thetas]
    
    # Create a full 'loop' so the first and last points are the same - Polygon
    xys.append(xys[0])
    
    
    xs = [xy[0] for xy in xys]
    ys = [xy[1] for xy in xys]
    
    return xs, ys

GetCircumferenceXY([0,0], 2, 3)

([0.0, 0.015637707705740515, -0.015637707705740488, 0.0],
 [0.01796615600585938,
  -0.009028434753417969,
  -0.009028434753417969,
  0.01796615600585938])

### Well Concept Relative Position
i.e. Are Well Concepts within specified radii around a point?

In [40]:
def BinByDistances(pointsToRank, centre, distBins):
    '''Well Concepts are ranked according to binned distance from platform. i.e. 
    Given input distances [0,2,5], the function will return a list of rankings where:
    0 indicates Well Concept is more than 5 km from platform
    1 indictates Well Concept is between 2 and 5 km from platform... etc
    '''
    #Radii must be ordered, i.e. [0,2,5]
    distBins = np.sort(distBins)
    
    num_bins = len(distBins)

    #calculate distances of all well concepts from platform 
    distances = [distance.distance(point_i, centre) for point_i in pointsToRank] #Karney 2013 distance
    
    distances = np.reshape(distances,[-1])

    
    #Reverse ranking - where 0 is closest to platform
    revranks = np.digitize(distances, distBins, right = False) #i.e. includes left bound = 0

    # Ranking - where 0 is furthest from platform
    ranking = [num_bins - revrank for revrank in revranks]
    
    return ranking

In [54]:
a = BinByDistances(tempwc, [0,0], [0,2,2.2])
b = BinByDistances(tempwc, [0,4], [0,200,300])
print([max(i) for i in zip(a,b)])

[2, 0, 1, 2, 2]


In [35]:
dct_colors = {0: "#808080",
          1:"#FF8000",
          2:"#00CC00"}

In [275]:
def GetWCRanks(wcpoints, plat_points,radii):
    '''
    Given the rank of Well Concepts, this derives the appropriate color for them
    
    If a WC has a rank of 3 (i.e. very close) with a platform, but a rank of 0 (i.e. far) from another platform, we 
    only care about the ranking of 3
    '''
    #Subset the WCs to only those which are on the creaming curve
    #A list for each platform, in which each WC is given a rank/bin
    WC_rank_by_platform = []
    
    for plat_point in plat_points:
        WC_rank_by_platform.append(BinByDistances([(a[0], a[1]) for a in wcpoints],plat_point,radii) )

    WC_best_rank = [max(WC_ranks)   for WC_ranks in zip(*WC_rank_by_platform)]
    
    
    return WC_best_rank  

## Map Construction

https://towardsdatascience.com/walkthrough-mapping-basics-with-bokeh-and-geopandas-in-python-43f40aa5b7e9

In [276]:
import json
from bokeh.models.widgets import Slider

from bokeh.io import output_file, show, save

from bokeh.models import (CDSView, ColorBar, ColumnDataSource,
                          CustomJS, CustomJSFilter, 
                          GeoJSONDataSource, HoverTool, LabelSet,
                          LinearColorMapper, Slider)
from bokeh.layouts import column, row, widgetbox
from bokeh.palettes import brewer
from bokeh.plotting import figure
# Input GeoJSON source that contains features for plotting
from bokeh.models.markers import SquareCross


from bokeh.layouts import column, row, WidgetBox
from bokeh.models import Panel
from bokeh.models.widgets import Tabs

In [307]:
lon_min = -60.64
lon_max = -60.1
lat_min = 9.85
lat_max = 10.5
n_concepts = 100

WC_name, WC_points, WC_resource = GenerateWCs(gdf_targPoly, n_concepts, lon_min, lon_max, lat_min, lat_max)

#Defaults
radiusinner=1
radiusouter=1.1


plat_points = [[-60.32, 10.07], [-60.335, 10.105]]
plat_name = ["Platform Roger 29", "Platform Quartz 5"]


#Any calculations based on the above defaults

_ = GetWCRanks(WC_points,plat_points, [0,radiusinner,radiusouter])
WC_colors = [dct_colors[wcrank] for wcrank in _]  




innerstepoutsx = []
innerstepoutsy = []

for plat_point in plat_points:
    innerstepoutx, innerstepouty = GetCircumferenceXY(plat_point,radiusinner , 401)
    innerstepoutsx.append(np.array(innerstepoutx[:-1]))
    innerstepoutsy.append(np.array(innerstepouty[:-1]))

outerstepoutsx = []
outerstepoutsy = []

for plat_point in plat_points:
    outerstepoutx, outerstepouty = GetCircumferenceXY(plat_point,radiusouter , 401)
    outerstepoutsx.append(np.array(outerstepoutx[:-1]))
    outerstepoutsy.append(np.array(outerstepouty[:-1]))
    


### Bokeh Map

In [315]:


# Create figure object.
p = figure(title = 'Polygons', 
           plot_height = 600 ,
           plot_width = 950, 
           toolbar_location = 'below',
           tools = "pan, wheel_zoom, box_zoom, reset",
           active_scroll = "wheel_zoom",
            x_range=(lon_min, lon_max),
            y_range=(lat_min, lat_max),          
          )
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None



# TARGET LAYER #
# Data Source
ds_targ = GeoJSONDataSource(geojson = gdf_targPoly[['target_id', 'target_name', 'polygon']].to_json())

# Bokeh Layer
p_targ = p.patches('xs','ys', source = ds_targ,
                   fill_color = "lightskyblue",
                   line_color = "blue", 
                   line_width = 0.1, 
                   fill_alpha = 0.2)


#PLATFORM LAYER #
platx = [a[0] for a in plat_points]
platy = [a[1] for a in plat_points]


# Data Source 
ds_plat = ColumnDataSource({'x':platx, 'y':platy, 'name': plat_name})
# Bokeh Layer
p_plat = p.diamond_cross("x", "y", size=20, source=ds_plat,
         line_color="black", fill_alpha=0.8)
# Label layer
labels = LabelSet(x="x", y="y", text="name", y_offset=8,
                  text_font_size="8pt", text_color="#555555",
                  source=ds_plat, text_align='center')
p.add_layout(labels)





#PLATFORM INNER STEP-OUT INNER #
#Data Source
source_step_inner = ColumnDataSource({'x':innerstepoutsx, 'y':innerstepoutsy})

#Bokeh layer
p_step_inner = p.patches('x','y', source = source_step_inner,
                   fill_color = None,
                   line_color = "#00CC00", 
                   line_width = 2, 
                   fill_alpha = 1)

#PLATFORM STEP-OUT OUTER #
source_step_outer = ColumnDataSource({'x':outerstepoutsx, 'y':outerstepoutsy})

p_step_outer = p.patches('x','y', source = source_step_outer,
                   fill_color = None,
                   line_color = "#FF8000", 
                   line_width = 1, 
                   fill_alpha = 0.1)

# WELL CONCEPTS
wcx = [a[0] for a in WC_points]
wcy = [a[1] for a in WC_points]
ds_wc = ColumnDataSource({'x':wcx, 'y':wcy, 'name':WC_name, 'resource':WC_resource, 'hex': WC_colors})

p_wc = p.circle("x", "y", size=5, source=ds_wc,
         color='hex', line_color="black", fill_alpha=0.8)

labels = LabelSet(x="x", y="y", text="name", y_offset=8,
                  text_font_size="8pt", text_color="#555555",
                  source=ds_wc, text_align='center')
p.add_layout(labels)

# Create hover tool
p.add_tools(HoverTool(renderers = [p_wc],
                      tooltips = [('Well Concept','@name'),
                                ('Resource','@resource')]))

### MPL Bar Chart - Creaming Curve

In [316]:
#Subset the WCs to only those which are on the creaming curve
WC_rank = GetWCRanks([(a[0], a[1]) for a in WC_points],plat_points,[0,radiusinner, radiusouter])

#Construct dataframe of all well concepts
df_WC = pd.DataFrame({"Well Concept": WC_name,"Resource": WC_resource, "Rank":WC_rank})

#Reduce to df of well concepts on Creaming Curve (i.e. within the radii)
df_WC_cc = df_WC[df_WC['Rank'] != 0].copy()

#Map ranks to colors
df_WC_cc.loc[:,'Color'] = df_WC_cc['Rank'].map(lambda x: dct_colors[x])

df_WC_cc.sort_values(by="Resource", inplace=True, ascending=False)

b = figure(x_range=df_WC_cc['Well Concept'], plot_height=250, title="Creaming Curve")
b.vbar(x=df_WC_cc['Well Concept'], top=df_WC_cc['Resource'], color=df_WC_cc['Color'], width=0.9)

b.xgrid.grid_line_color = None
b.y_range.start = 0



### Create Bokeh Layout

In [317]:
output_file("p.html")

# Create a row, with control panel and plot
layout = row(p, b)

# Make a tab with the layout 
tab = Panel(child=layout, title = 'Geospatial PoC')
tabs = Tabs(tabs=[tab])

save(tabs)

'/Users/Pri.balachandran@ibm.com/Desktop/BP/20191106_PlatformWCAnalysis/p.html'

In [221]:
N = 9
x = np.linspace(-2, 2, N)
y = x**2

xpts = np.array([-.09, -.12, .0, .12,  .09])
ypts = np.array([-.1,   .02, .1, .02, -.1])




xs=[xpts*(1+i/10.0)+xx for i, xx in enumerate(x)]
ys=[ypts*(1+i/10.0)+yy for i, yy in enumerate(y)]

In [243]:
xs

[array([-2.09, -2.12, -2.  , -1.88, -1.91]),
 array([-1.599, -1.632, -1.5  , -1.368, -1.401]),
 array([-1.108, -1.144, -1.   , -0.856, -0.892]),
 array([-0.617, -0.656, -0.5  , -0.344, -0.383]),
 array([-0.126, -0.168,  0.   ,  0.168,  0.126]),
 array([0.365, 0.32 , 0.5  , 0.68 , 0.635]),
 array([0.856, 0.808, 1.   , 1.192, 1.144]),
 array([1.347, 1.296, 1.5  , 1.704, 1.653]),
 array([1.838, 1.784, 2.   , 2.216, 2.162])]

In [245]:
stepoutsx[1]

array([-60.335     , -60.33471645, -60.3344336 , -60.33415217,
       -60.33387283, -60.33359625, -60.33332304, -60.33305377,
       -60.33278902, -60.33252928, -60.33227495, -60.33202645,
       -60.33178409, -60.3315482 , -60.33131894, -60.3310965 ,
       -60.330881  , -60.33067255, -60.33047118, -60.33027683,
       -60.33008955, -60.32990919, -60.32973567, -60.32956891,
       -60.32940869, -60.32925497, -60.32910748, -60.32896612,
       -60.32883062, -60.32870086, -60.32857663, -60.32845764,
       -60.32834383, -60.32823489, -60.32813073, -60.32803108,
       -60.32793576, -60.32784457, -60.32775739, -60.327674  ,
       -60.32759429, -60.32751804, -60.32744516, -60.3273754 ,
       -60.32730874, -60.32724489, -60.32718393, -60.32712551,
       -60.32706967, -60.32701628, -60.32696513, -60.32691622,
       -60.32686945, -60.32682464, -60.3267818 , -60.32674076,
       -60.32670155, -60.32666403, -60.32662805, -60.32659367,
       -60.32656079, -60.32652932, -60.32649924, -60.32